In [1]:
import os
import sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.optim as optim
import math
from tensorboardX import SummaryWriter

import import_ipynb
from CustomDataset import ControlsDataset
from Model import ConvNet

REPORT_EVERY_ITER = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device', device)

importing Jupyter notebook from CustomDataset.ipynb
   img  steeringAng
0    1            9
1    2            9
2    3            9
3    4            9
4    5            9
5    6            9
6    7            9
7    8            9
8    9            9
9   10            9
importing Jupyter notebook from Model.ipynb
   img  steeringAng
0    1            9
1    2            9
2    3            9
3    4            9
4    5            9
5    6            9
6    7            9
7    8            9
8    9            9
9   10            9
('units after conv', 512)
('conv parameters: ', 168224)
('fc parameters: ', 166804)
('input', torch.Size([256, 3, 480, 640]))
tensor([[-0.9283, -1.3190,  2.2029,  ...,  0.3183, -0.6540, -0.7787],
        [-0.7625, -1.2429,  2.1065,  ...,  0.3452, -0.5638, -0.7715],
        [-1.7469, -2.0672,  1.8136,  ..., -0.0621, -0.8134, -0.9749],
        ...,
        [-1.6539, -1.4375,  1.8011,  ..., -0.1056, -0.9806, -0.5534],
        [-1.1725, -1.2008,  1.5482,  ...,  0.

In [2]:
dataset = ControlsDataset()
dataset.convertTOClass()
print("Stack size", dataset.stack_size)
# dataloader = DataLoader(dataset, batch_size = 256, shuffle = True, num_workers = 0)
net = ConvNet().to(device)

print("Data size", dataset.data_frame.size)
print("number of parameters: ", sum(p.numel() for p in net.parameters()))

('Stack size', 1)
('units after conv', 512)
('conv parameters: ', 168224)
('fc parameters: ', 166804)
('Data size', 22036)
('number of parameters: ', 335028)


In [3]:
torch.cuda.empty_cache()

In [4]:
print(dataset.data_frame.head(10))

   img  steeringAng
0    1            9
1    2            9
2    3            9
3    4            9
4    5            9
5    6            9
6    7            9
7    8            9
8    9            9
9   10            9


In [5]:
validation_split = .2
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

# spliting the dataset
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


# Training data loader # NOTE had to remove shuffle
dataloader = DataLoader(dataset, batch_size = 256, num_workers = 0, sampler=train_sampler)

# Validation data loader # NOTE had to remove shuffle
validLoader = DataLoader(dataset, batch_size = 256, num_workers = 0, sampler=valid_sampler)

print("Total training stacks", len(dataloader))
print("Total validation stacks",len(validLoader))

('Total training stacks', 35)
('Total validation stacks', 9)


In [6]:
epochs = 25
optimizer = optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss() # Changed from Mean-Squared to Cross-Entropy
writer = SummaryWriter()
iter_no = 0

torch.cuda.empty_cache()
for epoch in range(epochs):
    for i_batch, sampled_batch in enumerate(dataloader):
        iter_no += 1
        images = sampled_batch['image'].to(device).float()
        controls = sampled_batch['control'].to(device).long()
        print("imagesSize: ", images.size(), "controlsSize:", controls.size())
        optimizer.zero_grad()
        prediction = net(images)
        
        controls = torch.flatten(controls)
        print(prediction.size(), controls.size())
        loss = criterion(prediction, controls)
        loss.backward()
        optimizer.step()

        if iter_no % REPORT_EVERY_ITER == 0:
            writer.add_scalar("Loss", loss.item(), iter_no)
            torch.save(net.state_dict(), "snapshots/saved_model_{}".format(loss.item()))
        
        out = "{0},{1}\tLoss:{2}\tAllocated:{3}GB\tCached:{4}GB\n"
        print(out.format(str(epoch),
                        str(iter_no),
                        round(loss.item(),5),
                         'na', 'na'
#                         round(torch.cuda.memory_allocated(0)/1024**3,3),
#                         round(torch.cuda.memory_allocated(0)/1024**3,3)
                        ))
        torch.cuda.empty_cache()

('imagesSize: ', torch.Size([256, 3, 480, 640]), 'controlsSize:', torch.Size([256, 1]))
tensor([[ 0.1306, -0.1476,  2.7605,  ..., -0.7274, -1.7934,  0.3676],
        [ 0.3611,  0.4607,  3.6346,  ..., -1.0026, -2.0363,  0.3889],
        [ 0.4955,  0.1366,  3.0700,  ..., -1.2756, -1.7462,  0.1997],
        ...,
        [ 0.2576,  0.0906,  3.1452,  ..., -1.1252, -1.7429,  0.2163],
        [ 0.5677,  0.3912,  3.9142,  ..., -1.0691, -3.5141, -0.7693],
        [ 0.2896,  0.7397,  3.5749,  ..., -1.0281, -1.8536,  0.5783]],
       grad_fn=<AddmmBackward>)
(torch.Size([256, 20]), torch.Size([256]))


IndexError: Target 4584 is out of bounds.

In [ ]:
# Validation Test
iter_no = 0
for i_batch, sampled_batch in enumerate(validLoader):
    iter_no += 1
    images = sampled_batch['image'].to(device).float()
    controls = sampled_batch['control'].to(device).float()

    optimizer.zero_grad()
    prediction = net(images)
    loss = criterion(prediction, controls)
    out = "{0},{1}\tLoss:{2}\tAllocated:{3}GB\tCached:{4}GB\n"
    print(out.format(str(epoch),
                    str(iter_no),
                    round(loss.item(),5),
                     'na', 'na'
#                         round(torch.cuda.memory_allocated(0)/1024**3,3),
#                         round(torch.cuda.memory_allocated(0)/1024**3,3)
                    ))
    torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024))